In [ ]:
!pip install biopython scikit-learn imbalanced-learn matplotlib seaborn --quiet
import pandas as pd
import numpy as np
from Bio import SeqIO
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from random import shuffle
import sklearn
from packaging import version
import seaborn as sns
import matplotlib.pyplot as plt
def load_sequences(file_paths, window=150, stride=100):
    sequences, labels = [], []
    for label, path in file_paths.items():
        for record in SeqIO.parse(path, "fasta"):
            seq = str(record.seq).replace("*", "")
            for i in range(0, len(seq) - window + 1, stride):
                frag = seq[i:i+window]
                if len(frag) == window:
                    sequences.append(frag)
                    labels.append(label)
    combined = list(zip(sequences, labels))
    shuffle(combined)
    sequences, labels = zip(*combined)
    return list(sequences), list(labels)
def get_kmers(seq, k=2):
    return [seq[i:i+k] for i in range(len(seq)-k+1)]
def build_kmer_features(sequences, k=2):
    all_kmers = set()
    kmer_counts = []
    for seq in sequences:
        kmers = get_kmers(seq, k)
        count = Counter(kmers)
        kmer_counts.append(count)
        all_kmers.update(count)
    all_kmers = sorted(list(all_kmers))
    df = pd.DataFrame(0, index=np.arange(len(sequences)), columns=all_kmers)
    for i, count in enumerate(kmer_counts):
        for kmer, freq in count.items():
            df.at[i, kmer] = freq
    return df
file_paths = {
    "Ebola": "/content/drive/MyDrive/sagar/Ebola_200.fasta",
    "Influenza_A": "/content/drive/MyDrive/sagar/Influenza_A_200.fasta",
    "Influenza_B": "/content/drive/MyDrive/sagar/Influenza_B_200.fasta",
    "SARS_CoV_2": "/content/drive/MyDrive/sagar/SARS_CoV_2_200.fasta",
    "Tuberculosis": "/content/drive/MyDrive/sagar/Tuberculosis_200.fasta",
    "Zika": "/content/drive/MyDrive/sagar/Zika_200.fasta",
    "Normal": "/content/drive/MyDrive/sagar/Human_200.fasta"
}
data, labels = load_sequences(file_paths, window=150, stride=100)
print(f"Augmented sequences: {len(data)}")
X = build_kmer_features(data, k=2)
y = pd.factorize(pd.Series(labels))[0]
selector = SelectKBest(mutual_info_classif, k=90)
X_selected = selector.fit_transform(X, y)
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X_selected, y)
X_train, X_test, y_train, y_test = train_test_split(
    X_balanced, y_balanced, test_size=0.2, stratify=y_balanced, random_state=42
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
if version.parse(sklearn.__version__) >= version.parse("1.2"):
    adaboost_model = AdaBoostClassifier(
        estimator=DecisionTreeClassifier(max_depth=3),
        n_estimators=120,
        learning_rate=0.5,
        random_state=42
    )
else:
    adaboost_model = AdaBoostClassifier(
        base_estimator=DecisionTreeClassifier(max_depth=3),
        n_estimators=120,
        learning_rate=0.5,
        random_state=42
    )
adaboost_model.fit(X_train_scaled, y_train)
y_pred = adaboost_model.predict(X_test_scaled)
acc = accuracy_score(y_test, y_pred)
print(f"\n Final AdaBoost Accuracy: {acc * 100:.2f}%")
print("\n Classification Report:\n")
print(classification_report(y_test, y_pred))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 25.1 MB/s eta 0:00:00
Augmented sequences: 5146

 Final AdaBoost Accuracy: 94.49%

 Classification Report:

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       419
           1       0.85      0.95      0.90       419
           2       0.97      0.92      0.94       420
           3       0.93      0.90      0.92       420
           4       0.88      0.90      0.89       420
           5       1.00      0.96      0.98       420
           6       1.00      1.00      1.00       420

    accuracy                           0.94      2938
   macro avg       0.95      0.94      0.95      2938
weighted avg       0.95      0.94      0.95      2938

